<a href="https://colab.research.google.com/github/sumeyyedemir5/nlp-preprocessing_and_textRepresentation/blob/main/BERT_and_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###TRANSFORMERS TABANLI METİN TEMSİLİ


*   BERT : (Bidirectional Encoder Representation from Transformers) bir kelimeyi incelerken önceki ve sonraki kelimelere de bakar. Metin sınıflandırma, adlandırılmış varlık tanıma, soru yanıtlama gibi görevlerde kullanılmaktadır.
*   GPT : (Generative Pre-trained Transformer) Tek yönlü çalışır. Bir kelimeyi tahmin ederken önceki kelimelere dayanır (autoregressive).

**Attention:** modelin belirli girdi parçalarına farklı derecelerde dikkat göstermesini sağlar.Özellikle bir kelimenin diğer kelimelerle olan ilişkisini anlamak için kullanılır.
Sorgu (Query) ve Anahtar(Key) Çarpımı : Cümledeki kelimelerin birbiriyle olan ilişki skoru hesaplanır.

**Input Embedding :** Girdi verilerini sayısal formata çevirmek için kullanılan tekniktir.

**Multi Head Attention:** Birden fazla dikkat başlığı ile çalışır. Anlam ilişkisi, gramatik ilişki, cümledeki konumu gibi.

**Masked multi head attention :** Model sadece geçmiş bilgileri kullanarak tahminde bulunur. Gelecekteki kelimeleri tahmin etmez.

**Addition & Normalization  :** Dikkat katmanının çıktısına giriş verilerini ekler ve ardından verileri normalleştirir.

**Feed Forward (İleri Besleme):** Her kelimeye ayrı ayrı uygulanan bir sinir ağı katmanıdır. Her encoder ve decoder katmanında bulunan bir ağdır.

**Output Embedding :** Çıktı kelimelerini veya sembollerini sayısal vektörlere dönüştürür.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text = "Transformers are amazing for natural language processing."
#tokenizasyon
inputs  = tokenizer(text,return_tensors = "pt")
with torch.no_grad():
   #geri yayılım iptal edilmiş oldu. Ağırlıklar güncellenmeyecek.
   outputs = model(**inputs)

#çıkışlardan ilk tokenları alalım
last_hidden_state = outputs.last_hidden_state
first_token_embedding = last_hidden_state[0,0,:].numpy()

In [ ]:
print("metin temsili ilk token : ")
print(first_token_embedding)

In [ ]:
from collections import Counter
import nltk
nltk.download('punkt_tab')
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

corpus = [
    "I love they",
    "they love apple and banana",
    "she loves you",
    "They love me"
]

tokens = [word_tokenize(sentence.lower()) for sentence in corpus]
tokens
"""
[['i', 'love', 'they'],
 ['they', 'love', 'apple', 'and', 'banana'],
 ['she', 'loves', 'you'],
 ['they', 'love', 'me']]
 """

In [ ]:
bigrams = []
for token_list in tokens:
  bigrams.extend(list(ngrams(token_list,2)))

bigrams
"""
[('i', 'love'),
 ('love', 'they'),
 ('they', 'love'),
 ('love', 'apple'),
 ('apple', 'and'),
 ('and', 'banana'),
 ('she', 'loves'),
 ('loves', 'you'),
 ('they', 'love'),
 ('love', 'me')]
"""

In [ ]:
bigrams_freq = Counter(bigrams)
bigrams_freq
#kelimelerin bigramlarını oluşturur ve ard arda gelme sıklığını gösterir.

In [ ]:
trigrams = []
for token_list in tokens:
  trigrams.extend(list(ngrams(token_list,3)))

trigrams_freq = Counter(trigrams)
trigrams_freq

In [ ]:
bigram = ("they","love")
prob_apple = trigrams_freq[("they", "love", "apple")]/bigrams_freq[bigram]
prob_me = trigrams_freq[("they", "love", "me")]/bigrams_freq[bigram]

##Hidden Markow Model

In [ ]:
import nltk
from nltk.tag import hmm
train_data = [
    [("I","PRP"),("am","VBP"),("a","DT"),("student","NN")],
    [("you","PRP"),("are","VBP"),("a","DT"),("teacher","NN")]
]

trainer = hmm.HiddenMarkovModelTrainer()
hmm_tagger = trainer.train(train_data)
test_sentence = "I am a teacher".split()
tags = hmm_tagger.tag(test_sentence)
print("etiketli cümle :",tags)

In [ ]:
from nltk.corpus import conll2000
nltk.download("conll2000")
train_data = conll2000.tagged_sents("train.txt")
test_data = conll2000.tagged_sents("test.txt")

trainer = hmm.HiddenMarkovModelTrainer()
hmm_tagger = trainer.train(train_data)
test_sentence = "I like going to park".split()
tags = hmm_tagger.tag(test_sentence)
tags